In [73]:
import warnings
warnings.filterwarnings('once')

from mqdq import babble, elegy, rhyme_classes, rhyme

import numpy as np
import pandas as pd
import scipy as sp

import glob
import umap
import math
import re

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

In [2]:
nux = babble.Babbler.from_file('./corpus/OV-nux.xml', name='Nux')

In [42]:
rhyme_classes.Line

mqdq.rhyme_classes.Line

In [78]:
def find_fours(bab: babble.Babbler) -> list[rhyme_classes.Line]:
    fours: list[rhyme_classes.Line] = []
    for l in bab._syl_source():
        m = 0
        try:
            m = max([len(x) for x in re.findall(r'a+',''.join([w.syls[-1].translate(rhyme.DEMACRON)[-1] for w in l]))])
        except:
            pass
        if m >= 4:
            fours.append(l)
    return(fours)
            


In [134]:
def is_four(l: rhyme_classes.Line) -> bool:
    lasts = []
    for w in l:
        if not w.syls:
            continue
        if (w.syls[-1] == 'kwe' or w.syls[-1]=='_') and len(w.syls) > 1:
            lasts.append(w.syls[-2].translate(rhyme.DEMACRON)[-1])
        elif w.syls[-1].translate(rhyme.DEMACRON) == 'a_st':
            lasts.append('a')
        else:
            lasts.append(w.syls[-1].translate(rhyme.DEMACRON)[-1])
    return(lasts.count('a') >= 4)
            


In [119]:
l = am[0]._syl_source()[12]

In [120]:
lasts = []
for w in l:
    if (w.syls[-1] == 'kwe' or w.syls[-1]=='_') and len(w.syls) > 1:
        lasts.append(w.syls[-2].translate(rhyme.DEMACRON)[-1])
    else:
        lasts.append(w.syls[-1].translate(rhyme.DEMACRON)[-1])

In [123]:
lasts.count('a')

3

In [122]:
l


Line([Word(pre_punct='', syls=['Sunt'], post_punct='', mqdq=<word sy="1A" wb="CM">Sunt</word>, color='', best_match=0.0, best_word=None, lock_color=False),
      Word(pre_punct='', syls=['`ti', 'bi'], post_punct='', mqdq=<word sy="1b1c" wb="DI">tibi</word>, color='', best_match=0.0, best_word=None, lock_color=False),
      Word(pre_punct='', syls=['`man', 'ja'], post_punct=',', mqdq=<word sy="2A2b" wb="CF">magna,</word>, color='', best_match=0.0, best_word=None, lock_color=False),
      Word(pre_punct='', syls=['`pu', 'ēr'], post_punct=',', mqdq=<word sy="2c3A" wb="CM">puer,</word>, color='', best_match=0.0, best_word=None, lock_color=False),
      Word(pre_punct='', syls=['ni', 'mi', '`ūm', 'kwe'], post_punct='', mqdq=<word sy="3b3c4A4b" wb="CF">nimiumque</word>, color='', best_match=0.0, best_word=None, lock_color=False),
      Word(pre_punct='', syls=['po', '`ten', 'ti', 'a'], post_punct='', mqdq=<word sy="4c5A5b5c" wb="DI">potentia</word>, color='', best_match=0.0, best_word=None

In [54]:
for i, f in enumerate(find_fours(am[0])):
    print(f"{i}: {' '.join([w.mqdq.text for w in fours[0]])}")

In [56]:
am = babble.multi_bookbabs(sorted(glob.glob('corpus/OV-amo*.xml')), name="Am.")
ep = babble.bookbabs('corpus/OV-epis.xml', name="Ep.")


In [55]:
am[0].raw_source

[<line metre="H" name="1" pattern="DDDD">
 <word sy="1A1b" wb="CF">Arma</word>
 <word sy="1c2A" wb="CM">graui</word>
 <word sy="2b2c3A" wb="CM">numero</word>
 <word sy="3b3c4A4b4c" wb="DI">uiolentaque</word>
 <word sy="5A5b" wb="CF">bella</word>
 <word sy="5c6A6X">parabam</word>
 </line>,
 <line metre="P" name="2" pattern="DD-|DD-">
 <word sy="1A1b1c" wb="DI">Edere,</word>
 <word sy="2A2b2c3A" wb="CM">materia</word>
 <word sy="4A4b4c5A5b" wb="CF">conueniente</word>
 <word sy="5c6X">modis.</word>
 </line>,
 <line metre="H" name="3" pattern="DDSS">
 <word sy="1A" wb="CM">Par</word>
 <word sy="1b1c" wb="DI">erat</word>
 <word sy="2A2b2c3A" wb="CM">inferior</word>
 <word sy="3T4A" wb="CM">uersus;</word>
 <word sy="4T5A5b" wb="CF">risisse</word>
 <word sy="5c6A6X">Cupido</word>
 </line>,
 <line metre="P" name="4" pattern="DS-|DD-">
 <word sy="1A1b1c" wb="DI">Dicitur</word>
 <word mf="SY" sy="2A">atque</word>
 <word sy="2T3A" wb="CM">unum</word>
 <word sy="4A4b4c5A5b" wb="CF">surripuisse</wo

In [138]:
def fourinate(babs: list[babble.Babbler]) -> None:
    ll, fours = 0,0
    for b in babs:
        ll += len(b._syl_source())
        for i, l in enumerate(b._syl_source()):
            if is_four(l):
                print(f"{b.name}.{i+1}: {' '.join([w.mqdq.text for w in l])}")
                fours += 1
    print(f"{fours} found in {ll} lines ({fours/ll:.2%})")

In [150]:
fourinate(ep[:15])

Ep. 1.31: Atque aliquis posita monstrat fera proelia mensa
Ep. 2.16: Mersa foret cana naufraga puppis aqua.
Ep. 2.90: Fessaque Bistonia membra lauabis aqua.
Ep. 2.110: Munera multa dedi, multa datura fui,
Ep. 3.8: Culpa tua est, quamuis haec quoque culpa tua est.
Ep. 3.124: Cumque mea patria laus tua uicta iacet?
Ep. 4.135: Illa coit firma generis iunctura catena,
Ep. 4.156: Da ueniam fassae duraque corda doma.
Ep. 5.2: Ista Mycenaea littera facta manu.
Ep. 5.98: Causa pudenda tua est; iusta uir arma mouet.
Ep. 7.86: Vre; minor culpa poena futura mea est.
Ep. 9.136: Truncaque limosa tempora mersit aqua;
Ep. 11.69: Fictaque sacra facit dicitque precantia uerba;
Ep. 12.70: Aurea barbarica stat dea facta manu.
Ep. 12.130: Caesaque uirginea membra paterna manu?
Ep. 13.7: Oscula plura uiro mandataque plura dedissem,
Ep. 13.112: Languida laetitia soluar ab ipsa mea?
Ep. 15.40: Nulla futura tua est, nulla futura tua est.
Ep. 15.200: Lesbides, Aeolia nomina dicta lyra,
Ep. 15.209: Qui mea uerb

In [151]:
fourinate(ep[15:])

Ep. 16.104: Caerulea peterem quin mea uota uia.
Ep. 16.140: Nullaque de facie nescia terra tua est
Ep. 16.144: Famaque de forma paene maligna tua est;
Ep. 16.146: Et tua materia gloria uicta sua est.
Ep. 16.247: Prodita sunt, memini, tunica tua pectora laxa
Ep. 16.275: Non mea sunt summa leuiter districta sagitta
Ep. 16.346: Laesa neque est Colcha Thessala terra manu;
Ep. 17.188: Flamma recens parua sparsa resedit aqua.
Ep. 18.20: Cetera cum charta dextra locuta mea est.
Ep. 18.134: Non aliter multa quam uia pressa rota.
Ep. 19.10: Ponitis in uaria tempora longa mora;
Ep. 19.49: Paucaque cum tacta perfeci stamina terra,
Ep. 19.63: Multaque praeterea lingua reticenda modesta,
Ep. 19.106: Fataque sint culpa nostra priora tua!
Ep. 20.128: Eueniant; poena tuta sit illa mea.
Ep. 21.60: Qua mihi te iactes, gratia nulla tua est.
Ep. 21.171: Parce laboranti, picta dea laeta pharetra,
17 found in 1554 lines (1.09%)


In [153]:
fourinate([ep[15]])

Ep. 16.104: Caerulea peterem quin mea uota uia.
Ep. 16.140: Nullaque de facie nescia terra tua est
Ep. 16.144: Famaque de forma paene maligna tua est;
Ep. 16.146: Et tua materia gloria uicta sua est.
Ep. 16.247: Prodita sunt, memini, tunica tua pectora laxa
Ep. 16.275: Non mea sunt summa leuiter districta sagitta
Ep. 16.346: Laesa neque est Colcha Thessala terra manu;
7 found in 376 lines (1.86%)


In [140]:
fourinate(am)

Am. 1 3.20: Prouenient causa carmina digna sua.
Am. 1 4.12: Da mea nec tepidis uerba ferenda Notis.
Am. 1 5.9: Ecce, Corinna uenit tunica uelata recincta,
Am. 1 5.10: Candida diuidua colla tegente coma,
Am. 1 7.4: Flet mea uesana laesa puella manu.
Am. 1 8.77: Surda sit oranti tua ianua, laxa ferenti;
Am. 1 8.80: Vanescit culpa culpa repensa tua.
Am. 1 11.14: Cetera fert blanda cera notata manu.
Am. 1 14.43: Facta manu culpaque tua dispendia sentis;
Am. 1 15.36: Pocula Castalia plena ministret aqua,
Am. 2 3.10: Sunt tibi cum domina signa ferenda tua.
Am. 2 4.42: Leda fuit nigra conspicienda coma;
Am. 2 5.12: Cui petitur uicta palma cruenta rea.
Am. 2 5.58: Lingua tua est nostris, nostra recepta tuis.
Am. 2 6.6: Pro longa resonent carmina uestra tuba.
Am. 2 10.26: Decepta est opera nulla puella mea;
Am. 2 11.42: Ipsa tua moueas turgida uela manu.
Am. 2 13.4: Ira digna mea, sed cadit ira metu.
Am. 2 18.13: Sceptra tamen sumpsi curaque tragoedia nostra
Am. 3 2.81: Sunt dominae rata uota m

In [141]:
fourinate(prop)

Prop. 1 11.10: Paruula Lucrina cumba moretur aqua,
Prop. 1 11.17: Non quia perspecta non es mihi cognita fama,
Prop. 1 16.27: O utinam traiecta caua mea uocula rima
Prop. 1 18.14: Non ita saeua tamen uenerit ira mea,
Prop. 1 19.15: Quarum nulla tua fuerit mihi, Cynthia, forma
Prop. 2 1.34: Actiaque in Sacra currere rostra Via,
Prop. 2 3.10: (Lilia non domina sint magis alba mea;
Prop. 2 5.28: "Cynthia, forma potens: Cynthia, uerba leuis."
Prop. 2 6.32: Orgia sub tacita condita laetitia!
Prop. 2 7.12: Tibia, funesta tristior illa tuba!
Prop. 2 8.11: Munera quanta dedi uel qualia carmina feci!
Prop. 2 9.36: Siue ea causa grauis, siue ea causa leuis.
Prop. 2 10.8: Bella canam, quando scripta puella mea est.
Prop. 2 12.22: (Haec mea Musa leuis gloria magna tua est),
Prop. 2 14.25: Magna ego dona tua figam, Cytherea, columna,
Prop. 2 16.2: Maxima praeda tibi, maxima cura mihi.
Prop. 2 19.31: Quin ego in assidua metuam tua nomina lingua:
Prop. 2 24.14: Quaeque nitent Sacra uilia dona uia.
Pr

In [143]:
fourinate([nux])

Nux.22: Vuaque laesa domum laesaque baca uenit;
Nux.26: Ista Clytaemestra digna querela fuit.
Nux.90: Non a uicina puluerulenta uia est.
Nux.114: Spinaque uindicta cetera tuta sua est.
Nux.116: Missa petunt auida saxa proterua manu.
5 found in 182 lines (2.75%)


In [82]:
pon = babble.multi_bookbabs(sorted(glob.glob('corpus/OV-pon*.xml')), name="Pont.")

In [144]:
fourinate(pon)

Pont. 1 2.21: Portaque uix firma summouet arma sera.
Pont. 1 2.37: Et grauior longa fit mea poena mora.
Pont. 1 2.69: Lenia pro misera fac modo uerba fuga.
Pont. 1 2.107: Ossa nec a Scythica nostra premantur humo,
Pont. 1 2.123: Clausit et aeterna ciuica bella sera:
Pont. 1 3.60: Altera Sarmatica spicula missa manu.
Pont. 1 5.61: Cur ego sollicita poliam mea carmina cura?
Pont. 1 6.32: Liberaque a ferro crura futura putet.
Pont. 1 10.28: Membraque sunt cera pallidiora noua.
Pont. 2 2.41: Verbaque nostra fauens Romana ad numina perfer,
Pont. 2 6.12: Qua mea debuerit currere cumba uia?
Pont. 2 9.62: Lucida Pieria tendis in astra uia.
Pont. 2 9.69: Nec mea subiecta conuicta est gemma tabella
Pont. 3 1.88: Vtque ea non teneas, tuta repulsa tua est.
Pont. 3 3.67: Per mea tela, faces, et per mea tela, sagittas,
Pont. 3 4.66: Dicta putet! pro se Musa locuta mea est.
Pont. 3 4.110: Textaque fortuna diuitiora sua,
Pont. 3 6.7: Quanta sit in media clementia Caesaris ira,
Pont. 3 8.11: Femina pro

In [84]:
tr = babble.multi_bookbabs(sorted(glob.glob('corpus/OV-tri*.xml')), name="Tr.")

In [145]:
fourinate(tr)

Tr. 1 1.85: Et mea cumba semel uasta percussa procella
Tr. 1 1.128: Vltimus, a terra terra remota mea.
Tr. 1 2.84: Quodque sit a patria tam fuga tarda, queror.
Tr. 1 3.8: Torquerant longa pectora nostra mora.
Tr. 1 11.12: Omnis ab hac cura cura leuata mea est.
Tr. 2 1.127: Vita data est, citraque necem tua constitit ira,
Tr. 2 1.172: Ponat et in nitida laurea serta coma,
Tr. 2 1.186: Pars erit ex poena magna leuata mea.
Tr. 2 1.214: Et sunt notitia multa minora tua;
Tr. 2 1.313: At cur in nostra nimia est lasciuia Musa,
Tr. 2 1.354: (Vita uerecunda est, Musa iocosa mea)
Tr. 3 2.6: Quodque magis uita Musa iocata mea est:
Tr. 3 4.69: Quod quoniam est anceps, intra mea pectora quemque
Tr. 3 4.75: Et qua quisque potest, aliqua mala nostra leuate,
Tr. 3 5.19: Multaque praeterea manifestaque signa fauoris
Tr. 3 7.8: Nec mala tam longa nostra leuata mora:
Tr. 3 10.24: Vina, nec hausta meri, sed data frusta bibunt.
Tr. 3 11.33: Omnia uera puta mea crimina, nil sit in illis,
Tr. 3 14.9: Est fug

In [97]:
prop = babble.multi_bookbabs(sorted(glob.glob('corpus/PROP-ele*.xml')), name="Prop.")

Prop. 1 16.26: O utinam traiecta caua mea uocula rima
Prop. 2 24.13: Quaeque nitent Sacra uilia dona uia.
Prop. 2 26.0: Vidi te in somnis fracta, mea uita, carina
Prop. 2 33.15: Cur tibi tam longa Roma petita uia?
Prop. 3 1.17: Detulit intacta pagina nostra uia.
Prop. 3 5.1: Stant mihi cum domina proelia dura mea.
Prop. 3 16.27: Me tegat arborea deuia terra coma,
Prop. 3 19.20: Tuque o Minoa uenumdata, Scylla, figura,
Prop. 3 19.21: Tondes purpurea regna paterna coma.
Prop. 4 1.73: Poscis ab inuita uerba pigenda lyra.
Prop. 4 10.19: Et galea hirsuta compta lupina iuba.
Prop. 4 11.25: Et iaceat tacita laxa catena sera.


In [136]:
ll, fours = 0,0
for b in prop:
    ll += len(b._syl_source())
    for i, l in enumerate(b._syl_source()):
        if is_four(l):
            print(f"{b.name}.{i+1}: {' '.join([w.mqdq.text for w in l])}")
            fours += 1
print(f"{fours} found in {ll} lines ({fours/ll:.2%})")

Prop. 1 11.10: Paruula Lucrina cumba moretur aqua,
Prop. 1 11.17: Non quia perspecta non es mihi cognita fama,
Prop. 1 16.27: O utinam traiecta caua mea uocula rima
Prop. 1 18.14: Non ita saeua tamen uenerit ira mea,
Prop. 1 19.15: Quarum nulla tua fuerit mihi, Cynthia, forma
Prop. 2 1.34: Actiaque in Sacra currere rostra Via,
Prop. 2 3.10: (Lilia non domina sint magis alba mea;
Prop. 2 5.28: "Cynthia, forma potens: Cynthia, uerba leuis."
Prop. 2 6.32: Orgia sub tacita condita laetitia!
Prop. 2 7.12: Tibia, funesta tristior illa tuba!
Prop. 2 8.11: Munera quanta dedi uel qualia carmina feci!
Prop. 2 9.36: Siue ea causa grauis, siue ea causa leuis.
Prop. 2 10.8: Bella canam, quando scripta puella mea est.
Prop. 2 12.22: (Haec mea Musa leuis gloria magna tua est),
Prop. 2 14.25: Magna ego dona tua figam, Cytherea, columna,
Prop. 2 16.2: Maxima praeda tibi, maxima cura mihi.
Prop. 2 19.31: Quin ego in assidua metuam tua nomina lingua:
Prop. 2 24.14: Quaeque nitent Sacra uilia dona uia.
Pr

In [103]:
ll, fours = 0,0
for b in ep:
    ll += len(b._syl_source())
    for i, l in enumerate(b._syl_source()):
        if is_four(l):
            print(f"{b.name}.{i}: {' '.join([w.mqdq.text for w in l])}")
            fours += 1
print(f"{fours} found in {ll} lines ({fours/ll:.2%})")

Ep. 5.1: Ista Mycenaea littera facta manu.
Ep. 15.199: Lesbides, Aeolia nomina dicta lyra,
Ep. 16.246: Prodita sunt, memini, tunica tua pectora laxa
Ep. 19.9: Ponitis in uaria tempora longa mora;
Ep. 19.62: Multaque praeterea lingua reticenda modesta,
Ep. 19.105: Fataque sint culpa nostra priora tua!
Ep. 21.170: Parce laboranti, picta dea laeta pharetra,
7 found in 3952 lines (0.18%)


In [146]:
tib = babble.multi_bookbabs(sorted(glob.glob('corpus/TIB-ele*.xml')), name="Tib.")

In [147]:
fourinate(tib)

Tib. 1 2.6: Clauditur et dura ianua firma sera.
Tib. 1 2.11: Et mala siqua tibi dixit dementia nostra,
Tib. 1 3.24: Illa tua totiens aera repulsa manu,
Tib. 1 5.14: Omina ter salsa deueneranda mola:
Tib. 1 6.28: Sobria supposita pocula uictor aqua.
Tib. 1 9.49: Illa uelim rapida Vulcanus carmina flamma
Tib. 1 10.26: Hostiaque e plena rustica porcus hara.
Tib. 1 10.40: Occupat in parua pigra senecta casa!
Tib. 2 1.16: Vinctaque post olea candida turba comas.
Tib. 2 3.51: Fictaque Cumana lubrica terra rota.
Tib. 2 3.79: Nunc si clausa mea est, si copia rara uidendi,
11 found in 1241 lines (0.89%)


In [99]:
cat = babble.bookbabs('corpus/CATVLL-carm.xml', name="Cat.")
cat_ele = [x for x in cat if x.elegiac and len(x) > 20]

In [148]:
fourinate(cat_ele)

Cat. 66.85: Illius a! mala dona leuis bibat irrita puluis;
Cat. 68.23: Omnia tecum una perierunt gaudia nostra,
Cat. 68.94: Omnia tecum una perierunt gaudia nostra,
Cat. 68.112: Tempore quo certa Stymphalia monstra sagitta
Cat. 68.142: Nec tamen illa mihi dextra deducta paterna
5 found in 351 lines (1.42%)
